# PePR: Membership Inference Attack (MIA)
Attack a single target model with a singe attack configuration.


## Prepare Environment
**Important: Restart the Runtime after this Cell!**
The restart is needed because of `pip install ./ml-pepr`.

In [1]:
!git clone https://github.com/hallojs/ml-pepr.git
%pip install ./ml-pepr
%pip install pylatex

Cloning into 'ml-pepr'...
remote: Enumerating objects: 1324, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 1324 (delta 113), reused 98 (delta 98), pack-reused 1187 (from 1)
Receiving objects: 100% (1324/1324), 2.13 MiB | 5.15 MiB/s, done.
Resolving deltas: 100% (839/839), done.
Processing ./ml-pepr
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.9/37.9 MB 35.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 52.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
cd ml-pepr/

/content/ml-pepr


In [27]:
pip install .

Processing /content/ml-pepr
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached adversarial_robustness_toolbox-1.6.2-py3-none-any.whl.metadata (10 kB)
  Using cached matplotlib-3.3.4.tar.gz (37.9 MB)
  Preparing metadata (setup.py) ... done
  Using cached numpy-1.19.5.zip (7.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached PyLaTeX-1.4.1.tar.gz (84 kB)
  Preparing metadata (setup.py) ... done
  Using cached PyYAML-5.4.1.tar.gz (175 kB)
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-

## Imports

In [5]:
from pepr.privacy import mia

import tensorflow as tf
from tensorflow.keras import optimizers

import numpy as np
import logging

## Setup Logging

In [6]:
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s', '%Y-%m-%d %H:%M:%S')

# TensorFlow Logger
file_handler_tf = logging.FileHandler('tf.log')
file_handler_tf.setLevel(logging.INFO)
file_handler_tf.setFormatter(formatter)

tf.get_logger().setLevel(logging.INFO)
logger_tf = tf.get_logger()
logger_tf.addHandler(file_handler_tf)

# PePR Logger
file_handler_pr = logging.FileHandler('pepr.privacy.mia.log')
file_handler_pr.setLevel(logging.DEBUG)
file_handler_pr.setFormatter(formatter)

stream_handler_pr = logging.StreamHandler()
stream_handler_pr.setLevel(logging.DEBUG)
stream_handler_pr.setFormatter(formatter)

logger_pr = logging.getLogger('pepr.privacy.mia')
logger_pr.addHandler(file_handler_pr)
logger_pr.addHandler(stream_handler_pr)

## Functions
Functions for creating models and preparing the dataset.

In [10]:
def get_target_model(input_shape, number_of_labels):
    target_model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation="tanh", padding='same', input_shape=input_shape),
        tf.keras.layers.MaxPool2D((2,2)),
        tf.keras.layers.Conv2D(64, (3,3), activation="tanh", padding='same'),
        tf.keras.layers.MaxPool2D((2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation="tanh"),
        tf.keras.layers.Dense(number_of_labels),
        tf.keras.layers.Softmax()
    ])
    return target_model

def get_attack_model(number_of_labels):
    attack_model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation="relu", input_shape=(number_of_labels,)),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ])
    return attack_model

def create_compile_shadow_model():
    """Create compiled target/shadow model.

    Returns
    -------
    tensorflow.python.keras.engine.sequential.Sequential
        A compiled tensorflow model.
    """
    input_shape = (32, 32, 3)
    number_classes = 100

    model = get_target_model(input_shape, number_classes)

    optimizer = optimizers.Adam(0.0001)
    loss = 'sparse_categorical_crossentropy'
    metrics = ["accuracy"]
    model.compile(optimizer, loss=loss, metrics=metrics)

    return model

def create_compile_attack_model():
    """Create compiled attack model.

    Returns
    -------
    tensorflow.python.keras.engine.sequential.Sequential
        A compiled tensorflow model.
    """
    number_classes = 100

    model = get_attack_model(number_classes)

    optimizer = optimizers.Adam(lr=0.0001)
    loss = 'binary_crossentropy'
    metrics = ["accuracy"]
    model.compile(optimizer, loss=loss, metrics=metrics)

    return model

def load_cifar100():
    """Loads and preprocesses the CIFAR100 dataset.

    Returns
    -------
    tuple
        (training data, training labels, test data, test labels)
    """
    train, test = tf.keras.datasets.cifar100.load_data()
    train_data, train_labels = train
    test_data, test_labels = test

    # Normalize the data to a range between 0 and 1
    train_data = np.array(train_data, dtype=np.float32) / 255
    test_data = np.array(test_data, dtype=np.float32) / 255

    # Reshape the images to (32, 32, 3)
    train_data = train_data.reshape(train_data.shape[0], 32, 32, 3)
    test_data = test_data.reshape(test_data.shape[0], 32, 32, 3)

    train_labels = np.reshape(np.array(train_labels, dtype=np.int32), (train_labels.shape[0],))
    test_labels = np.reshape(np.array(test_labels, dtype=np.int32), (test_labels.shape[0],))

    return np.vstack((train_data, test_data)), np.hstack((train_labels, test_labels))

## Init Dataset

In [11]:
data, labels = load_cifar100()

In [21]:
ls

Dockerfile-doc     docs/             notebooks/            pypiREADME.rst  requirements.txt  tf.log
Dockerfile-tf-cpu  LICENSE           pepr/                 pyproject.toml  setup.cfg
Dockerfile-tf-gpu  mlpepr.egg-info/  pepr.privacy.mia.log  README.rst      tests/


## Train Target Model

In [28]:
target_model = create_compile_shadow_model()
target_model.fit(data[40000:50000],
                 labels[40000:50000],
                 epochs=100,
                 batch_size=50,
                 verbose=0)
target_model.save('data/target_model')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=data/target_model.

## Run the Attack

In [26]:
attack_pars = {
    'number_classes': 100,
    'number_shadow_models': 100,
    'shadow_training_set_size': 2500,
    'create_compile_shadow_model': create_compile_shadow_model,
    'create_compile_attack_model': create_compile_attack_model,
    'shadow_epochs': 100,
    'shadow_batch_size': 50,
    'attack_epochs': 50,
    'attack_batch_size': 50,
}

# single target
data_conf = {
    'shadow_indices': list(range(40000)),
    'target_indices': list(range(40000, 50000)),
    'evaluation_indices': list(range(40000, 60000)),
    'record_indices_per_target': np.array([np.arange(10000)])
}

mia_attack = mia.Mia("MIA", attack_pars, data, labels, data_conf, [target_model])

mia_attack.run(save_path="data")

mia_attack.create_attack_report()

2024-12-11 14:01:24 - pepr.privacy.mia - INFO - Create mapping of records to shadow models.
INFO:pepr.privacy.mia:Create mapping of records to shadow models.
2024-12-11 14:01:24 - pepr.privacy.mia - INFO - Save mapping of records to shadow models: data/shadow_data_indices.npy.
INFO:pepr.privacy.mia:Save mapping of records to shadow models: data/shadow_data_indices.npy.


FileNotFoundError: [Errno 2] No such file or directory: 'data/shadow_data_indices.npy'

### Save Generated Data and Report
To download the `data` or `mia_report` directory from Google Colab, it needs to be compressed as a .zip file.

In [ ]:
!zip -r -q data.zip data

In [ ]:
!zip -r -q report.zip mia_report